# 22. 딕셔너리와 튜플보다는 헬퍼 클래스로 관리하자

- 파이썬 내장 딕셔너리 type : 객체의 수명이 지속되는 동안 __동적인__ 내부상태를 관리하는 용도로 좋음.
- __동적__ : 예상하지 못한 식별자들을 관리 해야 하는 상황
- 어떤 학생 집단의 성적을 기록해 보자. 학생별로 미리 정의 된 속성을 사용하지 않고 딕셔너리에 이름을 저장하는(key) 클래스를 정의

In [10]:
class SimpleGradebook(object):
    def __init__(self):
        self._grades = {}    # 객체생성시 성적 기록을 위한 딕셔너리를 담는다
    
    def add_student(self, name):
        self._grades[name] = []    # 처음 이름입력시에 이름을 키캆으로 점수를 담을 리스트를 생성
    
    def report_grades(self, name, score):
        self._grades[name].append(score)    # 해당 학생 이름에 점수를 기입해 주면 딕셔너리 키 값에 있는 리스트에 점수를 append해줌
        
    def average_grade(self, name):
        grades = self._grades[name]    # 함수 호출시 해당 딕셔너리 이름안에 점수의 평균을 return
        return sum(grades)/len(grades)

사용방법

In [11]:
book = SimpleGradebook()
book.add_student('Sangho')
book.report_grade('Sangho', 28)
print(book.average_grade('Sangho'))

AttributeError: 'SimpleGradebook' object has no attribute 'report_grade'

### 좀 더 나아가 과목별 점수도 추가하고 싶다 -> 어떤 구조를 써야할까?
 - 딕셔너리 키값 안에 다시 딕셔너리를 구성하여 key:과목이름 / values:점수로 매핑한다.
 - __이중 딕서너리 구조__ : {'이름1': {'과목명1':'성적1','성적2', '과목명2':'성적1','성적2'}, '이름2':...}

In [12]:
class BySubjectGradebook(object):
    def __init__(self):
        self._grades = {}    
        
    def add_student(self, name):
        self._grades[name] = {}    # 이름 딕셔너리 안에 성적별 점수를 담을 딕서너리 생성
        
    def report_grade(self, name, subject, grade):    # 이중 딕서너리 구조라 처리가 약간 복잡해짐
        by_subject = self._grades[name]
        grade_list = by_subject.setdefault(subject, [])   # setdefault : get과 비슷, dict에 key가 없으면 빈리스트를 반환
        grade_list.append(grade)
    
    def average_grade(self, name):
        by_subject = self._grades[name]    # 과목별 평균이 아니라 토탈 평균
        total, count = 0, 0
        for grades in by_subject.values():
            total += sum(grades)
            count += len(grades)
        return float(total / count)

사용 방법

In [13]:
book = BySubjectGradebook()
book.add_student('Sangho')
book.report_grade('Sangho', 'Math', 90)
book.report_grade('Sangho', 'Math', 90)
book.report_grade('Sangho', 'Gym', 100)
book.report_grade('Sangho', 'Gym', 80)

In [14]:
print(book.average_grade('Sangho'))

90.0


### 중간고사 보다 기말고사 비중을 높게 만드려면 어떻게 해야 할까?
- 내부 딕셔너리를 과목(키), 성적(값)에 매핑하지 않고, 성적과 비중을 담은 튜플에 매핑

In [17]:
class WeightedGradebook(object):
    
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = {}
        
    def report_gragde(self, name, subject, score, weight):
        by_subject = self_grades[name]
        grage_list = by_subject.setdefault(subject, [])
        grage_list.append((score, weight))    # 리스트 안에 단순한 값을 담는게 아니라 튜플 구조를 담는다.
        
    def average_grade(self, name):
        by_subject = self._grades[name]
        score_sum, score_count = 0, 0
        for subject, score in by_subject.items():
            subject_avg, total_weight = 0,0
            for score, weight in scores:    # 평균을 구할때 이중 루프가 생겨 이해가 어려운 코드가 됨
                #...
        return (score_sum/score_count)

IndentationError: expected an indented block (<ipython-input-17-c375b8b9c443>, line 21)

사용방법도 어려워 짐 -> 위치 인수 안에 있는 숫자들의 의미도 불 명확

In [ ]:
book.report_grade('Sangho', 'Math', 80, 0.1)

### 이렇게 복잡해질 경우 딕셔너리와 튜플대신 -> 클래스 계층 구조를 사용하자

- 처음만 해도 복잡한 헬퍼 클래스를 쓸 필요가 없었으나 계층이 증가할 수록 구조는 복잡해 진다.
- 계층이 한 단계가 넘어가는 중첩은 피해라(딕셔너리 안에 딕셔너리는 피해라)
- 인터페이스와 실제 구현 사이에 추상화 계층을 만들수 있다.

## 클래스 리팩토링

제일 안쪽 성적부터 옮기기 -> 클래스는 무겁다(튜플로)

In [ ]:
grades = []
grades.append([90, 0.45])
# ...
total = sum(score * weight for score, weight in grades)
total_weight = sum(weight for _, weight in grades)
average_grade = total / total_weight

더 많은 정보가 있을 수도 있으니 좀 더 유연하게 구조를 짠다.

In [ ]:
grades = []
grades.append([90, 0.45, 'good job'])
# ...
total = sum(score * weight for score, weight,_ in grades)  #'_' 은 관례적으로 사용하지 않을 변수에 입력
total_weight = sum(weight for _, weight,_ in grades)
average_grade = total / total_weight

튜플의 아이템이 2개가 넘어가면 다른 방법을 고려하자

#### namedtuple(immutable data class)
- 불변 데이터 클래스 : 위치 인수나 키워드 인수로 생성할 수 있음
- 필드는 이름이 붙은 속성으로 접근 가능
- 이름이 붙은 속성이 있으면 나중에 요구 사항이 또 변해도 namedtuple에서 직접 작성한 클래스에서 쉽게 바꿀 수 있음

#### namedtuple 제약
1. 기본 인수값 설정 불가능, 데이터에 선택적인 속성이 많으면 힘듦, 속성을 사용할 때는 클래스를 직접 정의하는게 나을 수도 있음.
2. ?

In [31]:
import collections
Grade = collections.namedtuple('Grade', ('score', 'weight'))

### class1. 단일 과목을 표현하는 클래스

In [32]:
class Subject(object):
    def __init__(self):
        self._grades = []
        
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
    
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total/total_weight

### class2. 한 학생이 공부한 과목을 표현하는 클래스

In [36]:
class Student(object):
    def __init__(self):
        self._subjects = {}
    
    def subjects(self, name):
        if name not in self._subjects:
            self._subjects[name] = Subject()
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count

### class3. 학생의 이름을 키로 사용해 동적으로 모든 학생을 담을 컨테이너 구축

In [37]:
class Gradebook(object):
    def __init__(self):
        self._student = {}
    
    def student(self, name):
        if name not in self._student:
            self._student[name] = Student()
        return self._student[name]

In [38]:
book = Gradebook()
sangho = book.student('Nohsangho')
math = sangho.subjects('Math')
math.report_grade(80, 0.3)
# ...

print(sangho.average_grade())

80.0


## 핵심정리
- 다른 딕셔너리나 긴 튜플을 값으로 담은 딕셔너리를 생성하지 말자
- 정식 클래스의 유연성이 필요 없다면 가벼운 불변 데이터 컨테이너에는 namedtuple을 사용하자
- 내부 상태를 관리하는 딕셔너리가 북잡해 지면 여러 헬퍼 클래스를 사용하는 방식으로 관리 코드를 바꾸자